In [1]:
import pandas as pd
from nltk import WhitespaceTokenizer
from nltk.corpus import stopwords, words, wordnet
from nltk.lm import Vocabulary
from collections import OrderedDict
import numpy as np
import torch
from torch import nn
import concurrent.futures
from torch.optim import Adam
from data_loading import process_text_df, NewsText, tensorize_sentences, collate_fn
from tqdm import tqdm as pbar
from models import NewsNet
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch import tensor
import matplotlib.pyplot as plt
import tqdm
stopwords = stopwords.words()
words = words.words() 
wordnet = wordnet.words()

In [2]:
fake, true = pd.read_csv('Fake.csv'), pd.read_csv('True.csv')
fake['label'] = 'fake'
true['label'] = 'true'
news = pd.concat((fake, true))
news = news.sample(frac=1)
news.reset_index(inplace=True, drop=True)

In [3]:
p1 = process_text_df(news, ['title', 'text'])

In [4]:
%%time
p1.process_text_col()

CPU times: user 2.97 s, sys: 1.21 s, total: 4.18 s
Wall time: 4min 23s


In [5]:
p1.df

,title,text,subject,date,label
0,"[south, korea, call, bonenumb, sanction, north...","[seoul, reuter, , south, korea, warn, north, k...",politicsNews,"January 13, 2016",true
1,"[trump, tri, scare, us, isi, terrorist, group,...","[know, trump, paranoia, isi, make, realli, ann...",News,"February 11, 2017",fake
2,"[trump, vietnam, presid, underscor, free, open...","[reuter, , us, presid, donald, trump, vietname...",politicsNews,"November 12, 2017",true
3,"[obamacar, privaci, practic, question, critic,...","[rep, diann, black, healthcar, industri, decad...",Government News,"Jun 15, 2015",fake
4,"[clinton, trounc, trump, gop, district, doubl,...","[donald, trump, continu, sink, polit, parti, d...",News,"August 10, 2016",fake
...,...,...,...,...,...
44893,"[trump, america, first, speech, alarm, us, alli]","[london, reuter, , donald, trump, first, major...",politicsNews,"April 27, 2016",true
44894,"[saudi, arabia, forfeit, sovereign, right, enr...","[riyadh, reuter, , saudi, arabia, forfeit, sov...",worldnews,"December 21, 2017",true
44895,"[trump, new, pathet, antihillari, frame, bill,...","[anyon, point, finger, mistreat, women, probab...",News,"May 23, 2016",fake
44896,"[lebanon, pass, disput, tax, hike, fund, publi...","[beirut, reuter, , lebanon, parliament, approv...",worldnews,"October 9, 2017",true


In [6]:
p1.tokenize_sentences()

In [7]:
l, labs = tensorize_sentences(p1.df.text.apply(lambda sent: sent[:50]), p1.df.label)

In [ ]:
net = NewsNet(p1.vocab, hidden_size=4, embedding_dim=8, num_layers=2)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = Adam(net.parameters(), lr=1e-4, weight_decay=5e-6)
idx = len(l) // 4

l_tr, labs_tr = l[:-2 * idx], labs[: -2 * idx]
l_val, labs_val = l[-2 * idx:-idx], labs[-2 * idx:-idx]
l_tst, labs_tst = l[-idx:], labs[-idx:]
l_tr, l_val, labs_tr, labs_val = l[:idx], l[idx:], labs[:idx], labs[idx:]
tr_set = NewsText(l_tr, labs_tr)
val_set = NewsText(l_val, labs_val)
_, val_set = enumerate(DataLoader(val_set, batch_size=len(val_set), collate_fn=collate_fn)).__next__()
val_features, val_labels = val_set
val_labels = val_labels.unsqueeze(-1).float()
loader = DataLoader(tr_set, batch_size=4, collate_fn=collate_fn)
loss_list = []
val_loss_list = []
accuracy_list = []
epochs = 60
for i in range(epochs):
    print(f'Epoch {i + 1}')
    for sents, labels in pbar(loader):
        net.train()
        labels = labels.float().unsqueeze(-1)
        out = net(sents)
        loss = loss_fn(out, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        net.eval()
    with torch.no_grad():
        val_out = net(val_features)
        val_loss = loss_fn(val_out, val_labels)
        val_loss_list.append(val_loss.item())
        val_guesses = torch.round(nn.Sigmoid()(val_out))
        accuracy = (val_guesses == val_labels).float().mean().item()
        
        accuracy_list.append(accuracy)
        
    loss_list.append(loss.item())

  0%|          | 5/2806 [00:00<00:56, 49.69it/s]

Epoch 1


  0%|          | 6/2806 [00:00<00:47, 58.72it/s]

Epoch 2


  0%|          | 6/2806 [00:00<00:47, 59.32it/s]

...

  0%|          | 7/2806 [00:00<00:46, 60.15it/s]

Epoch 59


  0%|          | 6/2806 [00:00<00:49, 56.29it/s]

Epoch 60


 34%|███▍      | 963/2806 [00:16<00:29, 63.50it/s]

In [ ]:
fig, ax = plt.subplots()
pd.Series(loss_list).plot(ax=ax, label='Tr')
pd.Series(val_loss_list).plot(ax=ax, label='Val')
fig.set_size_inches(fig.get_size_inches() * 1.5)
ax.legend()

In [ ]:
pd.Series(accuracy_list).plot()

In [ ]:
tst_set = NewsText(l_tst, labs_tst)
_, tst_set = enumerate(DataLoader(tst_set, batch_size=len(tst_set), collate_fn=collate_fn)).__next__()
tst_features, tst_labels = tst_set
tst_labels = tst_labels.unsqueeze(-1).float()

In [ ]:
with torch.no_grad():
    tst_out = net(tst_features)
    tst_loss = loss_fn(tst_out, tst_labels)
    tst_guesses = torch.round(nn.Sigmoid()(tst_out))
    accuracy_tst = (tst_guesses == tst_labels).float().mean().item()

In [ ]:
accuracy_tst